# Generating Representative Sentences from Web Crawling Data

In this notebook, we try to implement methods that generating sentences from source-articles and term-frequency data.

## DATA API
- PTT
    + Deployeed on: http://52.26.156.12:8000/
    + Two functions:
        * get_json: return json as a string
            - http://52.26.156.12:8000/get_json/?item_type=termscores&item_date=2021-11-09
            - http://52.26.156.12:8000/get_json/?item_type=posts&item_date=2021-11-09
        * get_file: return file object (.csv)

In [1]:
import requests
import json

posts = requests.get('http://52.26.156.12:4001/get_json/?item_type=posts')
tfreq = requests.get('http://52.26.156.12:4001/get_json/?item_type=termscores')

print(posts)

<Response [200]>


In [2]:
import pandas as pd

posts = pd.DataFrame(eval(posts.text))
print(posts.head())

C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\tsyo\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


                   id                     title        author  \
0  M.1638436945.A.7B3  [新聞] 綠名嘴：林秉樞有計畫滲透本土陣營 涉國  knight791211   
1  M.1638400512.A.D63   [新聞] 林秉樞被收押再爆「高嘉瑜應該很開心啦        lycppt   
2  M.1638430101.A.78B  [新聞] 林秉樞嗆殺前女友判決公開了！案號「新北        jackhg   
3  M.1638452388.A.277   [新聞] 快訊／林口無預警大停電　居民崩潰：洗      GETpoint   
4  M.1638416465.A.9AF  [新聞] 高嘉瑜遭家暴登國際版面　南韓媒體報導、        zuvupa   

                                                link  \
0  https://www.ptt.cc/bbs/Gossiping/M.1638436945....   
1  https://www.ptt.cc/bbs/Gossiping/M.1638400512....   
2  https://www.ptt.cc/bbs/Gossiping/M.1638430101....   
3  https://www.ptt.cc/bbs/Gossiping/M.1638452388....   
4  https://www.ptt.cc/bbs/Gossiping/M.1638416465....   

                                             content  n_comments  n_push  \
0  \n備註請放最後面 違者新聞文章刪除\n\n1.媒體來源:\n聯合新聞網\n\n2.記者署名...        1223     739   
1  \n蘋果日報\n林秉樞被收押再爆「高嘉瑜應該很開心啦！」　很多人在罵總統「她也很開心啊！」\...        1074     568   
2  \n\n1.媒體來源:自由時報\n\n\n2.記者署名:記者吳政峰／台北報導\n\

In [3]:
tfreq = pd.DataFrame(eval(tfreq.text))
print(tfreq.head(10))

  term      score
0   林秉  32.518803
1  高嘉瑜  27.970257
2   台北  21.824998
3  吳政峰  13.799446
4   台灣  11.920594
5  太魯閣  11.911675
6  張君豪  11.473145
7   黃男  11.434055
8  帥哥寶  11.090355
9  加拿大  11.047396


In [4]:
print(posts['content'].iloc[0])


備註請放最後面 違者新聞文章刪除

1.媒體來源:
聯合新聞網

2.記者署名:
記者林政忠／即時報導


3.完整新聞標題:
綠名嘴：林秉樞有計畫滲透本土陣營 涉國安問題

4.完整新聞內文:
民進黨立委高嘉瑜遭男友林秉樞施暴，林秉樞被各界起底。名嘴溫朗東今天在臉書貼文指
出，林秉樞冒充國安高層、長期住在容易交付資金跟資料的飯店，甚至2018年9月30日在
思想坦克發表文章，主張台灣有揮舞五星旗的自由。

溫朗東質疑，林秉樞2013年的碩士論文指導教授是經常往返兩岸、提倡中國配偶權益的楊
君仁；林在政大政治博士班做中國研究，網路專欄、對綠營政治人物的政治獻金，都在
2018年中啟動，開始密集向綠營捐款，「林秉樞是有計畫性的滲透進本土陣營，這是標準
的國安問題」。


5.完整新聞連結 (或短網址):



In [5]:
def article_to_sentences(article, min_length=5):
    ''' Parse an article into a list sentences. '''
    # Define sentence-break symbols
    bs = ['\n','，','。','；','！','？','「','」','.',':','（','）','／','　','~','：']
    # Loop through the article character-by-character
    sentences = []
    tmp = []
    for char in article:
        if not char in bs:
            tmp.append(char)
        else:
            if len(tmp)>=min_length:
                sentences.append(''.join(tmp).strip())
            tmp = []
    return(sentences)

sentences = article_to_sentences(posts['content'].iloc[0])
print(sentences)

['備註請放最後面 違者新聞文章刪除', '聯合新聞網', '記者林政忠', '完整新聞標題', '林秉樞有計畫滲透本土陣營 涉國安問題', '完整新聞內文', '民進黨立委高嘉瑜遭男友林秉樞施暴', '林秉樞被各界起底', '名嘴溫朗東今天在臉書貼文指', '林秉樞冒充國安高層、長期住在容易交付資金跟資料的飯店', '甚至2018年9月30日在', '思想坦克發表文章', '主張台灣有揮舞五星旗的自由', '溫朗東質疑', '林秉樞2013年的碩士論文指導教授是經常往返兩岸、提倡中國配偶權益的楊', '林在政大政治博士班做中國研究', '網路專欄、對綠營政治人物的政治獻金', '2018年中啟動', '開始密集向綠營捐款', '林秉樞是有計畫性的滲透進本土陣營', '的國安問題', '完整新聞連結 (或短網址)']


In [6]:
from sentence_transformers import SentenceTransformer, util

SENTENCE_TRANSFORMER_PATH = 'D:/workspace/language_models/distiluse-base-multilingual-cased-v2/'
st = SentenceTransformer(SENTENCE_TRANSFORMER_PATH)

N_TERMS = 10

term_embeddings = st.encode(list(tfreq['term'].iloc[:N_TERMS]))
term_weights = list(tfreq['score'].iloc[:N_TERMS])

def evaluate_sentence_embedding(se, term_embeddings, term_weights):
    import numpy as np
    score = 0
    for i in range(len(term_embeddings)):
        score+=np.dot(se, term_embeddings[i])*term_weights[i]
    score = score/sum(term_weights)
    return(score)

sent_scores = []
for s in sentences:
    se = st.encode(s)
    sent_scores.append(evaluate_sentence_embedding(se, term_embeddings, term_weights))
    
results = pd.DataFrame({'sentence':sentences, 'score':sent_scores})
print(results)

                               sentence     score
0                      備註請放最後面 違者新聞文章刪除  0.006235
1                                 聯合新聞網  0.117250
2                                 記者林政忠  0.140703
3                                完整新聞標題  0.075216
4                    林秉樞有計畫滲透本土陣營 涉國安問題  0.022502
5                                完整新聞內文  0.075176
6                      民進黨立委高嘉瑜遭男友林秉樞施暴  0.035229
7                              林秉樞被各界起底  0.049957
8                         名嘴溫朗東今天在臉書貼文指  0.037363
9            林秉樞冒充國安高層、長期住在容易交付資金跟資料的飯店  0.019265
10                        甚至2018年9月30日在  0.049918
11                             思想坦克發表文章  0.042990
12                        主張台灣有揮舞五星旗的自由  0.067028
13                                溫朗東質疑  0.152365
14  林秉樞2013年的碩士論文指導教授是經常往返兩岸、提倡中國配偶權益的楊  0.020723
15                       林在政大政治博士班做中國研究  0.071705
16                    網路專欄、對綠營政治人物的政治獻金  0.016970
17                             2018年中啟動  0.083009
18                            開始密集向綠營捐款  0.003944


In [7]:
print(results.sort_values('score', ascending=False))

                               sentence     score
13                                溫朗東質疑  0.152365
2                                 記者林政忠  0.140703
1                                 聯合新聞網  0.117250
20                                的國安問題  0.093048
17                             2018年中啟動  0.083009
3                                完整新聞標題  0.075216
5                                完整新聞內文  0.075176
15                       林在政大政治博士班做中國研究  0.071705
12                        主張台灣有揮舞五星旗的自由  0.067028
7                              林秉樞被各界起底  0.049957
10                        甚至2018年9月30日在  0.049918
11                             思想坦克發表文章  0.042990
8                         名嘴溫朗東今天在臉書貼文指  0.037363
6                      民進黨立委高嘉瑜遭男友林秉樞施暴  0.035229
19                     林秉樞是有計畫性的滲透進本土陣營  0.026503
4                    林秉樞有計畫滲透本土陣營 涉國安問題  0.022502
14  林秉樞2013年的碩士論文指導教授是經常往返兩岸、提倡中國配偶權益的楊  0.020723
9            林秉樞冒充國安高層、長期住在容易交付資金跟資料的飯店  0.019265
16                    網路專欄、對綠營政治人物的政治獻金  0.016970


In [9]:
stop_words = ['ETtoday','CrazyWinnie', 'ETTODAY', 'YAHOO新聞','聯合新聞網', '-----', '東森新聞','完整新聞標題','完整新聞內文']

sent = []
sent_scores = []
# Loop through articles
for i in range(3):
    article = posts['content'].iloc[i]
    sentences = article_to_sentences(article)
    # Loop through sentences
    for s in sentences:
        if not s in stop_words:
            se = st.encode(s)
            sent.append(s)
            sent_scores.append(evaluate_sentence_embedding(se, term_embeddings, term_weights))
    # Check total number of sentences
    if len(sent_scores)>=10:
        break

results = pd.DataFrame({'sentence':sent, 'score':sent_scores})
print(results.sort_values('score', ascending=False))

                               sentence     score
10                                溫朗東質疑  0.152365
1                                 記者林政忠  0.140703
17                                的國安問題  0.093048
14                             2018年中啟動  0.083009
12                       林在政大政治博士班做中國研究  0.071705
9                         主張台灣有揮舞五星旗的自由  0.067028
4                              林秉樞被各界起底  0.049957
7                         甚至2018年9月30日在  0.049918
8                              思想坦克發表文章  0.042990
5                         名嘴溫朗東今天在臉書貼文指  0.037363
3                      民進黨立委高嘉瑜遭男友林秉樞施暴  0.035229
16                     林秉樞是有計畫性的滲透進本土陣營  0.026503
2                    林秉樞有計畫滲透本土陣營 涉國安問題  0.022502
11  林秉樞2013年的碩士論文指導教授是經常往返兩岸、提倡中國配偶權益的楊  0.020723
6            林秉樞冒充國安高層、長期住在容易交付資金跟資料的飯店  0.019265
13                    網路專欄、對綠營政治人物的政治獻金  0.016970
18                        完整新聞連結 (或短網址)  0.016876
0                      備註請放最後面 違者新聞文章刪除  0.006235
15                            開始密集向綠營捐款  0.003944
